In [1]:
# get flux boundaries weights
# adjust new fluxes
# run for at least 10, 100, 1,000 samples: FBA, single and double KO

In [2]:
import cobra

In [3]:
import pandas, numpy, sys

In [38]:
import multiprocessing, multiprocessing.pool

In [4]:
flux_weights_file = '/Users/adrian/projects/riia/results/flux_change_results_file.csv'
model_file = '/Users/adrian/projects/riia/data/model/Recon3DModel_301.mat'

threads = 8

In [5]:
cobra_config = cobra.Configuration()
cobra_config.processes = threads
cobra.Configuration()

Attribute,Description,Value
solver,Mathematical optimization solver,glpk
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,8
cache_directory,Path for the model cache,/Users/adrian/Library/Caches/cobrapy
max_cache_size,Maximum cache size in bytes,104857600
cache_expiration,Model cache expiration time in seconds (if any),None


# 1. read information

## 1.1. read boundary weights

In [6]:
weights = pandas.read_csv(flux_weights_file, index_col=0)
print(weights.shape)
weights

(2636, 785)


,55293.1,3939.1,6566.1,1738.1,4967.2,4967.1,8050.1,128.1,2531.1,8604.1,...,1329.1,1350.1,9377.1,1351.1,1349.1,1337.1,1345.1,1327.1,1347.1,1340.1
GSM752709.cel,1.27,1.66,1.75,1.78,1.02,1.02,1.81,1.51,1.43,0.32,...,0.25,0.85,1.36,0.56,1.80,1.51,0.69,0.00,0.78,1.45
GSM752710.cel,1.21,1.42,1.74,1.66,0.97,0.97,1.63,1.02,0.94,0.26,...,0.35,0.57,1.34,0.94,1.74,1.31,0.69,0.13,0.46,1.80
GSM752711.cel,1.50,1.46,1.75,1.85,0.82,0.82,1.83,1.38,1.12,0.20,...,0.14,0.70,1.20,0.52,1.75,1.51,0.60,0.05,0.63,1.40
GSM752712.cel,1.44,1.46,1.69,1.82,0.94,0.94,1.79,1.42,1.21,0.17,...,0.22,0.68,1.17,0.53,1.65,1.38,0.82,0.14,0.66,1.42
GSM752713.cel,1.15,1.27,1.56,1.71,1.05,1.05,1.33,0.80,1.18,1.29,...,0.06,0.47,1.39,0.86,1.74,1.51,1.19,0.11,0.59,1.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM46391.CEL,0.12,1.00,1.96,0.61,1.08,1.08,0.81,1.82,0.72,0.86,...,1.41,1.44,0.62,0.54,1.26,0.64,1.05,1.80,1.19,0.30
GSM46392.CEL,0.05,1.80,0.77,0.49,1.53,1.53,1.21,1.77,0.93,1.17,...,2.00,1.45,0.87,1.44,1.26,0.81,1.45,1.77,1.21,1.26
GSM46393.CEL,0.05,1.53,0.66,0.52,1.81,1.81,0.88,1.69,0.64,1.29,...,1.94,1.42,0.74,1.41,1.13,0.93,0.57,1.78,1.00,0.97
GSM46394.CEL,0.04,0.14,1.99,0.04,0.53,0.53,0.27,1.93,1.83,1.87,...,1.32,1.11,0.57,0.19,1.01,0.43,0.32,1.62,1.04,0.24


## 1.2. read model

In [8]:
%%time
model = cobra.io.load_matlab_model(model_file)

CPU times: user 2min 37s, sys: 211 ms, total: 2min 38s
Wall time: 2min 38s


# 2. constraining

In [39]:
%%time

def boundary_setter(reaction, reaction_weights, unique_reaction_set):
    
    #print('\t', reaction_weights)
    #new_reaction_bounds = reaction.bounds
    
    # f.1. define presence of logic rules
    grr = reaction.gene_reaction_rule
    #print(grr)
    code = 0
    if 'or' in grr:
        code = code + 1
    if 'and' in grr:
        code = code + 2
      
    # f.2. procceed to change
    
    # if there are no rules
    if code == 0:
        w = reaction_weights.values[0]
        nlb = reaction.bounds[0]*w
        nub = reaction.bounds[1]*w
        new_reaction_bounds = (nlb, nub)
        
    # there are only OR rules
    elif code == 1: # the mean of all reaction genes. if no expression available, 1
        #print('or present')
        #print(reaction_weights.values)
        working_w = list(reaction_weights.values)
        #team = list(set([gene.id.split('.')[0] for gene in reaction.genes]))
        missing_elements = len(unique_reaction_set) - len(working_w)
        for element in range(missing_elements):
            working_w.append(1)
            #print('CASE')
        #print(working_w)
        w = numpy.mean(working_w)
        #print(w)
        nlb = reaction.bounds[0]*w
        nub = reaction.bounds[1]*w
        new_reaction_bounds = (nlb, nub)
        #print(new_reaction_bounds)
        
    # there are only AND rules
    elif code == 2:
        #print('and present')
        #print(reaction_weights.values)
        w = numpy.min(reaction_weights.values)
        nlb = reaction.bounds[0]*w
        nub = reaction.bounds[1]*w
        new_reaction_bounds = (nlb, nub)
        #print(new_reaction_bounds)
    
    # there are both AND and OR rules
    elif code == 3:
        #print('both present')
        #print(grr.split(' or '))
        all_cases = []
        or_cases = grr.split(' or ')
        isoforms = []
        for case in or_cases:
            if 'and' in case:
                and_cases = case.split(' and ')
                and_cases_cleanish = [element.replace('(', '') for element in and_cases]
                and_cases_cleaned = [element.replace(')', '') for element in and_cases_cleanish]
                #print('and cases', and_cases)
                #print('and cases', and_cases_cleaned)
                and_cases_bounds = []
                for element in and_cases_cleaned:
                    if element in reaction_weights:
                        and_cases_bounds.append(reaction_weights[element])
                        #print('gold', and_cases_bounds)
                    
                if len(and_cases_bounds) != 0:
                    z = numpy.min(and_cases_bounds)
                    all_cases.append(z)                    
            else:
                isoform_root = case.split('.')[0]
                if case in reaction_weights:
                    all_cases.append(reaction_weights[case])
                else:
                    if isoform_root not in isoforms:
                        all_cases.append(1)
                # adding at the level of isoform_root =
                isoforms.append(isoform_root)
                    
        w = numpy.mean(all_cases)
        nlb = reaction.bounds[0]*w
        nub = reaction.bounds[1]*w
        new_reaction_bounds = (nlb, nub)
    else:
        raise ValueError('logic rule not anticipated')
    
    return new_reaction_bounds

def constrain_evaluator(sampleID, model, flux_weights):
    
    print('working with sample {}...'.format(sampleID))
    
    #! iterate reactions
    with model as model:
        for reaction in model.reactions:
            
            # 1. define a new boundary for each reaction
            local_reaction_genes = [gene.id for gene in reaction.genes]
            unique_reaction_set = list(set([element.split('.')[0] for element in local_reaction_genes]))
            
            # 1.1. check if the reaction contains genes and if so, if they are ammenable to constraining
            if len(local_reaction_genes) >= 1:
                ammenable_genes = [element for element in local_reaction_genes if element in flux_weights.index]
                if len(ammenable_genes) >= 1:
                
                    # 1.2. actual constraining
                    reaction_weights = flux_weights[ammenable_genes]
                    new_reaction_bounds = boundary_setter(reaction, reaction_weights, unique_reaction_set)
                    reaction.bounds = new_reaction_bounds
                    
        #! 2. perform analysis
        model.optimize()
        result = model.objective.value
            
    return result

# serial
print('serial...')
for sampleID in weights.index[:8]:
    result = constrain_evaluator(sampleID, model, weights.loc[sampleID])
    print('\t {}'.format(result))
    
# parallel 
print('parallel...')

hydra = multiprocessing.pool.Pool(threads)
parallel_output = hydra.map(constrain_evaluator, weights.index[:64])

serial...
working with sample GSM752709.cel...
	 714.1473677871769
working with sample GSM752710.cel...
	 718.0651740711809
working with sample GSM752711.cel...
	 731.0004369938437
working with sample GSM752712.cel...
	 724.8891359533867
working with sample GSM752713.cel...
	 703.772766864273
working with sample GSM752714.cel...
	 724.7004564678936
working with sample GSM752715.cel...
	 699.6062922839728
working with sample GSM752716.cel...
	 704.6554121674941
parallel...


KeyboardInterrupt: 

In [1]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))


KeyboardInterrupt: 